# Installation des différents package

In [ ]:
!pip install semantic-kernel beautifulsoup4 python-dotenv google-cloud-bigquery pyyaml google-cloud-bigquery-storage python-docx ddgs google-cloud-storage

# Import des variables d'environnement

Cette section charge les variables d'environnement depuis le fichier .env. Ces variables contiennent les clés d'API et les identifiants nécessaires pour se connecter aux services Azure OpenAI, Google Cloud, etc.

**Important :** N'oubliez pas de copier-coller le fichier .env à la racine du code avec toutes les variables nécessaires.

In [ ]:
from dotenv import load_dotenv
import os
load_dotenv(override=True)

# Configuration des identifiants Google Cloud

Cette cellule configure le chemin vers le fichier de compte de service Google Cloud. Ce fichier est requis pour s'authentifier auprès des services Google Cloud comme BigQuery et Cloud Storage.

In [ ]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="colas-training-service-account.json"

# Import des librairies nécessaires

Cette cellule importe toutes les bibliothèques Python nécessaires pour le projet, incluant Semantic Kernel pour la création d'agents IA, les connecteurs Azure OpenAI, les outils pour BigQuery, ainsi que les bibliothèques utilitaires pour le traitement de données et les requêtes HTTP.

In [ ]:
import asyncio

from semantic_kernel import Kernel
from semantic_kernel.utils.logging import setup_logging
from semantic_kernel.functions import kernel_function
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior
from semantic_kernel.connectors.ai.chat_completion_client_base import ChatCompletionClientBase
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.functions.kernel_arguments import KernelArguments
import logging
from semantic_kernel.connectors.ai.open_ai.prompt_execution_settings.azure_chat_prompt_execution_settings import (
    AzureChatPromptExecutionSettings,
)
import yaml
import os
from semantic_kernel.functions import kernel_function
from datetime import datetime
import requests
from typing import Annotated, Optional, List, Dict, Any
from google.cloud import bigquery
import json

# Création de l'agent "Example"

## Tool Météo

Cette section définit le plugin météo qui permet à l'agent d'obtenir les conditions météorologiques actuelles et les prévisions pour des coordonnées géographiques spécifiques. Le plugin utilise l'API Open-Meteo pour récupérer les données de température, humidité, vent et précipitations.

In [ ]:
class WeatherPlugin:
    """Weather plugin for Semantic Kernel with Open-Meteo API integration."""

    @kernel_function(
    name="get_current_weather",
    description="Fetches current weather for given coordinates using Open-Meteo API"
    )
    def get_current_weather(
        self,
        latitude: float,
        longitude: float
    ) -> dict:
        """Fetches current weather for given coordinates.

        Args:
            latitude: Latitude coordinate
            longitude: Longitude coordinate

        Returns:
            dict: Weather details with status
        """
        url = "https://api.open-meteo.com/v1/forecast"

        params = {
            "latitude": latitude,
            "longitude": longitude,
            "current": [
                "temperature_2m",
                "relative_humidity_2m",
                "precipitation",
                "weather_code",
                "wind_speed_10m",
            ],
            "timezone": "auto",
        }

        try:
            response = requests.get(url, params=params)

            if response.status_code != 200:
                return {"status": "error", "error_message": f"API Error: {response.text}"}

            data = response.json()
            current = data["current"]

            weather_description = self._get_weather_description(current["weather_code"])

            return {
                "status": "success",
                "latitude": data["latitude"],
                "longitude": data["longitude"],
                "description": weather_description,
                "temperature": current["temperature_2m"],
                "humidity": current["relative_humidity_2m"],
                "wind_speed": current["wind_speed_10m"],
                "precipitation_1h": current["precipitation"],
            }
        except Exception as e:
            return {"status": "error", "error_message": f"Request failed: {str(e)}"}

    @kernel_function(
        name="get_weather_forecast",
        description="Fetches daily weather forecast for up to 16 days using Open-Meteo API"
    )
    def get_weather_forecast(
        self,
        latitude: float,
        longitude: float,
        days: int = 7
    ) -> dict:
        """Fetches daily weather forecast.

        Args:
            latitude: Latitude coordinate
            longitude: Longitude coordinate
            days: Number of days for forecast (default 7, max 16)

        Returns:
            dict: Forecast details with status
        """
        url = "https://api.open-meteo.com/v1/forecast"

        days = min(days, 16)

        params = {
            "latitude": latitude,
            "longitude": longitude,
            "daily": [
                "temperature_2m_max",
                "temperature_2m_min",
                "weather_code",
                "precipitation_sum",
                "wind_speed_10m_max",
                "wind_gusts_10m_max",
                "wind_direction_10m_dominant",
            ],
            "timezone": "auto",
            "forecast_days": days,
        }

        try:
            response = requests.get(url, params=params)

            if response.status_code != 200:
                return {"status": "error", "error_message": f"API Error: {response.text}"}

            data = response.json()
            daily = data["daily"]

            forecasts = []
            for i in range(len(daily["time"])):
                weather_description = self._get_weather_description(daily["weather_code"][i])
                forecasts.append(
                    {
                        "date": daily["time"][i],
                        "temp_max": daily["temperature_2m_max"][i],
                        "temp_min": daily["temperature_2m_min"][i],
                        "description": weather_description,
                        "precipitation": daily["precipitation_sum"][i],
                        "wind_speed_max": daily["wind_speed_10m_max"][i],
                        "wind_gusts_max": daily["wind_gusts_10m_max"][i],
                        "wind_direction": daily["wind_direction_10m_dominant"][i],
                    }
                )

            return {
                "status": "success",
                "latitude": data["latitude"],
                "longitude": data["longitude"],
                "forecasts": forecasts,
            }
        except Exception as e:
            return {"status": "error", "error_message": f"Request failed: {str(e)}"}

    @kernel_function(
        name="get_current_date",
        description="Fetches the current date and time"
    )
    def get_current_date(self) -> dict:
        """Fetches the current date and time.

        Returns:
            dict: Current date and time with status
        """
        try:
            current_date_time = datetime.utcnow().isoformat()
            return {"status": "success", "current_date_time": current_date_time}
        except Exception as e:
            return {"status": "error", "error_message": str(e)}

    def _get_weather_description(self, code: int) -> str:
        """Maps WMO weather code to description.

        Args:
            code: WMO weather code

        Returns:
            str: Weather description
        """
        weather_codes = {
            0: "clear sky",
            1: "mainly clear",
            2: "partly cloudy",
            3: "overcast",
            45: "fog",
            48: "depositing rime fog",
            51: "light drizzle",
            53: "moderate drizzle",
            55: "dense drizzle",
            56: "light freezing drizzle",
            57: "dense freezing drizzle",
            61: "slight rain",
            63: "moderate rain",
            65: "heavy rain",
            66: "light freezing rain",
            67: "heavy freezing rain",
            71: "slight snow",
            73: "moderate snow",
            75: "heavy snow",
            77: "snow grains",
            80: "slight rain showers",
            81: "moderate rain showers",
            82: "violent rain showers",
            85: "slight snow showers",
            86: "heavy snow showers",
            95: "thunderstorm",
            96: "thunderstorm with slight hail",
            99: "thunderstorm with heavy hail",
        }
        return weather_codes.get(code, "unknown")


## Tool Bigquery

Cette section définit le plugin BigQuery qui permet à l'agent d'interroger la base de données BigQuery pour récupérer des informations sur les chantiers. Le plugin peut exécuter des requêtes SQL SELECT de manière sécurisée et retourner les résultats au format JSON.

In [ ]:
from google.cloud import bigquery
from typing import Annotated, Optional
from semantic_kernel.functions import kernel_function
import json
import yaml


class BigQueryPlugin:
    """
    A simplified Semantic Kernel plugin for BigQuery table queries.
    The agent determines the SQL query, executes it, and returns results.
    """
    
    def __init__(self):
        """
        Initialize the BigQuery plugin.
        Configured for table: colas-training.colas_data.data-chantier
        """
        self.client = bigquery.Client(project="colas-training")
        self.project_id = "colas-training"
        self.dataset_id = "colas_data"
        self.table_name = "data-chantier"
        self.full_table_id = f"{self.project_id}.{self.dataset_id}.`{self.table_name}`"
        self.schema_cache = {}
    
    def _get_table_schema_yaml(self) -> str:
        """
        Get table schema in YAML format for the agent to understand the table structure.
        """
        if self.schema_cache:
            return self.schema_cache.get('schema', '')
        
        try:
            table_ref = f"{self.project_id}.{self.dataset_id}.{self.table_name}"
            table = self.client.get_table(table_ref)
            
            schema_dict = {
                'platform': 'BigQuery',
                'project': self.project_id,
                'dataset': self.dataset_id,
                'table': self.table_name,
                'full_path': f"{self.project_id}.{self.dataset_id}.{self.table_name}",
                'description': table.description or 'Colas construction site data',
                'columns': []
            }
            
            for field in table.schema:
                column_info = {
                    'name': field.name,
                    'type': field.field_type,
                }
                if field.description:
                    column_info['description'] = field.description
                schema_dict['columns'].append(column_info)
            
            yaml_schema = yaml.dump(schema_dict, default_flow_style=False)
            self.schema_cache['schema'] = yaml_schema
            return yaml_schema
            
        except Exception as e:
            return f"Error retrieving schema: {str(e)}"
    
    @kernel_function(
        name="get_table_schema",
        description="Gets the schema of the data-chantier table including column names, types, and descriptions. ALWAYS use this first to understand the table structure before creating a query."
    )
    async def get_table_schema(self) -> str:
        """Retrieve table schema so the agent can construct appropriate queries."""
        return self._get_table_schema_yaml()
    
    @kernel_function(
        name="execute_query",
        description="Executes a SQL SELECT query against the data-chantier table and returns the results. The table path is already configured as `colas-training.colas_data.data-chantier`."
    )
    async def execute_query(
        self,
        query: Annotated[str, "The SQL SELECT query to execute. The table is available as `colas-training.colas_data.data-chantier` or you can use backticks for the table name with hyphens."],
        max_results: Annotated[int, "Maximum number of results to return. Default is 100."] = 100
    ) -> str:
        """
        Execute a BigQuery SQL query and return results as JSON.
        """
        # Security: Ensure query is SELECT only
        query_upper = query.strip().upper()
        if not query_upper.startswith('SELECT'):
            return json.dumps({
                'error': 'Only SELECT queries are allowed',
                'row_count': 0,
                'data': []
            })
        
        # Prevent dangerous operations
        dangerous_keywords = ['DROP', 'DELETE', 'INSERT', 'UPDATE', 'CREATE', 'ALTER', 'TRUNCATE']
        if any(keyword in query_upper for keyword in dangerous_keywords):
            return json.dumps({
                'error': 'Query contains prohibited keywords. Only SELECT queries are allowed.',
                'row_count': 0,
                'data': []
            })
        
        try:
            query_job = self.client.query(query)
            results = query_job.result(max_results=max_results)
            
            # Convert to list of dictionaries
            rows = []
            for row in results:
                rows.append(dict(row))
            
            result_data = {
                'row_count': len(rows),
                'total_rows': results.total_rows,
                'data': rows,
                'success': True
            }
            
            return json.dumps(result_data, indent=2, default=str)
            
        except Exception as e:
            return json.dumps({
                'error': f"Error executing query: {str(e)}",
                'row_count': 0,
                'data': [],
                'success': False
            })

## Tool Date du Jour

Cette section définit le plugin de gestion des dates qui permet à l'agent d'obtenir la date et l'heure actuelles dans le fuseau horaire de Paris. Cet outil est essentiel pour calculer les dates relatives (demain, après-demain, etc.) dans les requêtes utilisateur.

In [ ]:
from datetime import datetime
from zoneinfo import ZoneInfo
from semantic_kernel.functions import kernel_function

class DateTimePlugin:
    """Plugin for fetching current date and time information."""
    
    @kernel_function(
        name="get_current_date",
        description="Fetches the current date and time in Paris timezone"
    )
    def get_current_date(self) -> dict:
        """Fetches the current date and time in Paris timezone.
        
        Returns:
            dict: A dictionary with 'status' and 'current_date_time'.
        """
        try:
            current_date_time = datetime.now(ZoneInfo("Europe/Paris")).isoformat()
            return {"status": "success", "current_date_time": current_date_time}
        except Exception as e:
            return {"status": "error", "error_message": str(e)}


## Chargement Règles Métier

Cette section charge les règles métier depuis un document Word stocké dans Google Cloud Storage. Ces règles définissent les critères de faisabilité des travaux de chantier en fonction des conditions météorologiques (température, vent, précipitations). Les règles sont ensuite intégrées dans le prompt de l'agent pour guider ses décisions.

In [ ]:
from google.cloud import storage
from google.oauth2 import service_account
from docx import Document
import io

def load_regles_chantier() -> str:
    """Load construction site rules from GCS document."""
    bucket_name = "colas-data"
    source_blob_name = "atelier-5-deepdive-gcp/critères_travaux_formation_5_gcp.docx"
    service_account_file = "colas-training-service-account.json"
    
    try:
        # Load credentials from service account JSON file
        credentials = service_account.Credentials.from_service_account_file(
            service_account_file
        )
        
        storage_client = storage.Client(credentials=credentials)
        bucket = storage_client.bucket(bucket_name)
        blob = bucket.blob(source_blob_name)
        
        in_memory_file = io.BytesIO()
        blob.download_to_file(in_memory_file)
        in_memory_file.seek(0)
        
        doc = Document(in_memory_file)
        full_text = [para.text for para in doc.paragraphs]
        return '\n'.join(full_text)
    except Exception as e:
        print(f"Error loading regles_chantier: {e}")
        return ""


regles_chantier = load_regles_chantier()
print(regles_chantier[:1000])

## Prompt de l'Agent

Cette section définit le prompt système de l'agent de faisabilité des chantiers. Le prompt décrit le rôle de l'agent, les actions qu'il doit effectuer (identification du chantier, requête BigQuery, récupération météo, évaluation de faisabilité), et les règles métier à appliquer. L'agent doit répondre en français avec des recommandations claires.

In [ ]:
agent_prompt = f"""
Role
You are the Chantier Feasibility Agent, an expert system designed to determine the operational viability of construction sites (chantiers) based on their type, location, and weather forecast.
Your main objective is to provide a clear, decisive, and actionable recommendation to the user regarding the feasibility of construction work on a requested date (within the next 7 days maximum).
Your final response must always be in French.

Actions
You must execute the following sequence of actions to answer every user request:

1. Identify Target and Date
   - Determine the TYPE_chantier (or the Chef de Chantier if specified) and the requested date.
   - **Date Calculation:** If the user provides a relative date (e.g., "tomorrow," "in 2 days," "après-demain"), you **MUST** call the `current_date_tool` to get today's date (November 7, 2025) and calculate the effective calendar date.
   - The final, calculated date must be within the next 7 days from today (November 7, 2025).

2. Query BigQuery (bigquery_tool)
   - Search for records in colas-training.colas_data.data-chantier matching the requested TYPE_chantier or the NOM_chef_chantier / ID_chef_chantier.
   - Possible outcomes:
       * **Single chantier found:** proceed with that chantier.
       * **Multiple chantiers found:** retrieve all matching chantiers (e.g., tous les chantiers gérés par le chef de chantier X) and process them individually.
       * **No chantier found:** inform the user that no matching chantier exists in the database.

3. Coordinate Processing
   - For each chantier, COOR_chantier is formatted as a string "latitude longitude".
   - Extract numeric values latitude and longitude to use them in the weather tool.

4. Weather Query (open_weather_tool)
   - For each chantier, use OpenWeatherApiTool to get weather data for its coordinates:
     - **Future dates (up to 7 days)**: call `perform_action(latitude, longitude, action_type='forecast', days=7)`.
     - **Today**: call `perform_action(latitude, longitude, action_type='current')`.
   - Identify the forecast corresponding to the user's requested date.
   - If the forecast for a chantier cannot be found, inform the user that the weather data is unavailable for that chantier.

5. Determine Feasibility
   - For each chantier, apply the feasibility rules (given below in French) to the obtained forecast data.
   - Mapping for rule evaluation:
       * Température → `temp_day` (°C)
       * Vent → `wind_speed` (m/s)
       * Précipitations → if `desc` includes "rain" or "shower", treat as precipitation > 0 mm.
   - Classify each chantier as one of three statuses:
       ✅ **FAISABLE** : toutes les conditions respectent les seuils.
       ⚠️ **VIGILANT** : conditions limites — chantier réalisable avec mesures de prévention.
       🚫 **NON FAISABLE** : au moins une condition dépasse les seuils critiques.

6. Safety & Prevention (for VIGILANT mode)
   - If a chantier is in "VIGILANT" mode, provide relevant safety and prevention recommendations based on the weather risk:
       * Vent fort → sécuriser le matériel, limiter travail en hauteur.
       * Température basse → pauses régulières, équipement thermique adapté.
       * Pluie → protéger le matériel, planifier activités sous abri.
       * Chaleur élevée → hydratation renforcée, travail aux heures fraîches.

7. Final Response (in French)
   - For each chantier, present a complete answer including:
       * Le type de chantier
       * La date demandée
       * Les conditions météo pertinentes (température, vent, précipitation, etc.)
       * Le verdict final : FAISABLE / VIGILANT / NON FAISABLE
       * Si VIGILANT → ajouter les règles de prévention/sécurité adaptées.
   - If multiple chantiers are requested, provide **individual results for each chantier** clearly separated.

   - Example phrasing for multiple chantiers:
       "Chantier 1 - Toiture du 10 novembre : FAISABLE. Température 9°C, Vent 12 km/h, Pas de précipitation."
       "Chantier 2 - Peinture Extérieure du 10 novembre : VIGILANT. Vent 28 km/h. Mesures recommandées : sécuriser les échafaudages, limiter le travail en hauteur."
       "Chantier 3 - Fondations du 10 novembre : NON FAISABLE. Pluie modérée prévue (2 mm/h)."

Context

BigQuery Table Structure
The table accessible via bigquery_tool is: colas-training.colas_data.data-chantier
Columns:
- ID_chef_chantier
- NOM_chef_chantier
- COOR_chantier (format: "latitude longitude")
- TYPE_chantier (e.g., Toiture, Fondations, Peinture Extérieure)

Feasibility Rules (Critical Operational Rules)
The feasibility of a chantier is strictly governed by the following French thresholds.
You must evaluate the weather data against these rules:
{regles_chantier if regles_chantier else "Rules not loaded. Please ensure regles_chantier is provided."}

Constraints
- Time Limit: Only dates ≤ 7 days from today (November 7, 2025) are allowed.
- Unlisted Types: If the chantier type is not in the feasibility table, reply:
  "Les règles de faisabilité pour ce type de chantier ne sont pas définies dans mon contexte. Je ne peux pas évaluer la viabilité."
"""

## Execution de l'agent

Cette section contient la fonction principale qui initialise le kernel Semantic Kernel, configure les services Azure OpenAI, enregistre tous les plugins (BigQuery, Météo, Datetime), et démarre une boucle de conversation interactive. L'utilisateur peut poser des questions sur la faisabilité des chantiers et l'agent répondra en utilisant les outils disponibles.

In [ ]:
os.getenv("DEPLOYMENT_NAME")

In [ ]:

async def main():
    # Initialize the kernel
    kernel = Kernel()

    # Add Azure OpenAI chat completion
    chat_completion = AzureChatCompletion(
        deployment_name=os.getenv("DEPLOYMENT_NAME"),
        api_key=os.getenv("API_KEY"),
        base_url=os.getenv("BASE_URL"),
    )
    kernel.add_service(chat_completion)

    # Set up detailed logging for function calls
    setup_logging()
    
    # Optional: Add console handler with formatting for better readability
    console_handler = logging.StreamHandler()
    console_handler.setLevel(logging.DEBUG)
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    console_handler.setFormatter(formatter)
    logging.getLogger("semantic_kernel").addHandler(console_handler)

    # Add BigQuery plugin
    bq_plugin = BigQueryPlugin()
    kernel.add_plugin(
        bq_plugin,
        plugin_name="BigQuery"
    )
    kernel.add_plugin(
    WeatherPlugin(),
    plugin_name="Meteo"
    )
    # Register the DateTime plugin
    kernel.add_plugin(
        DateTimePlugin(),
        plugin_name="Datetime"
    )

    # Enable planning
    execution_settings = AzureChatPromptExecutionSettings()
    execution_settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

    # Create a history of the conversation
    history = ChatHistory()

    history.add_system_message(agent_prompt)

    # Initiate a back-and-forth chat
    userInput = None
    while True:
        # Collect user input
        userInput = input("User > ")

        # Terminate the loop if the user says "exit"
        if userInput == "exit":
            break

        # Add user input to the history
        history.add_user_message(userInput)

        # Get the response from the AI
        result = await chat_completion.get_chat_message_content(
            chat_history=history,
            settings=execution_settings,
            kernel=kernel,
        )

        # Print the results
        print("Assistant > " + str(result))

        # Add the message from the agent to the chat history
        history.add_message(result)

await main()

# Création d'un Agent de A à Z

Cette section présente un nouveau projet : créer un agent qui calcule le coût en matériaux pour réaliser une surface spécifique. Le but de l'agent est de calculer le prix pour remplir une surface donnée avec un matériau précis.

**Exemple d'utilisation :** "Combien ça va me coûter de couler du béton sur 2 mètres de haut et 40 mètres de large ?"

## Outils Mathématiques

Cette section doit contenir un plugin d'outils mathématiques qui permettra à l'agent d'effectuer les calculs nécessaires pour déterminer les volumes, surfaces et coûts des matériaux. L'agent pourra ainsi calculer automatiquement les quantités de matériaux nécessaires en fonction des dimensions fournies.

In [ ]:
# Inclure votre code ici

## Outils Recherche Web ( Google )

Cette section définit le plugin de recherche web utilisant DuckDuckGo. Cet outil permet à l'agent de rechercher sur internet les prix actuels des matériaux de construction. Le plugin peut effectuer des recherches rapides ou récupérer le contenu complet des pages web pour extraire les informations de prix.

In [ ]:
import requests
import time

class GoogleSearchPlugin:
    def __init__(self, api_key: str, cx: str):
        self.api_key = api_key
        self.cx = cx

    @kernel_function(
        name="search",
        description="Search the web using Google Custom Search"
    )
    def search(self, query: str, max_results: int = 10) -> str:
        url = (
            "https://www.googleapis.com/customsearch/v1"
            f"?key={self.api_key}&cx={self.cx}"
            f"&q={query}&num={max_results}"
        )
        resp = requests.get(url)
        resp.raise_for_status()
        data = resp.json()

        items = data.get("items", [])
        if not items:
            return "No results found."

        lines = []
        for item in items:
            title = item.get('title')
            snippet = item.get('snippet')
            link = item.get('link')
            
            # Retry logic for 403 errors
            max_retries = 5
            retry_count = 0
            link_content = "Error fetching the content"
            
            while retry_count < max_retries:
                try:
                    headers = {
                        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
                    }
                    link_resp = requests.get(link, headers=headers, timeout=10)
                    
                    if link_resp.status_code == 403:
                        retry_count += 1
                        time.sleep(1)  # Wait before retrying
                        continue
                    
                    link_resp.raise_for_status()
                    link_content = link_resp.text[:5000]
                    break  # Success, exit retry loop
                    
                except requests.RequestException as e:
                    if retry_count < max_retries - 1:
                        retry_count += 1
                        time.sleep(1)
                    else:
                        link_content = f"Error fetching the content: {str(e)}"
                        break
            
            lines.append(
                f"- {title}\n  {snippet}\n  {link}\n  Content Preview: {link_content}\n"
            )

        return "\n".join(lines)

## Outils Recherche Web ( DuckDuckGo )

In [ ]:
from semantic_kernel.functions import kernel_function
from ddgs import DDGS
from ddgs.exceptions import DDGSException, RatelimitException, TimeoutException
import aiohttp
from bs4 import BeautifulSoup
from typing import Optional


class DuckDuckGoSearchPlugin:
    """Custom DuckDuckGo search plugin with webpage content fetching using ddgs library"""
    
    def __init__(self, proxy: Optional[str] = None, timeout: int = 10):
        """
        Initialize the plugin with ddgs library
        
        Args:
            proxy: Optional proxy string (e.g., "socks5h://127.0.0.1:9150" or "tb" for Tor Browser)
            timeout: Timeout for requests (default: 10)
        """
        self.proxy = proxy
        self.timeout = timeout
    
    @kernel_function(
        name="search_and_fetch",
        description="Search the web using DuckDuckGo and fetch full content from top results"
    )
    async def search_and_fetch(
        self,
        query: str,
        num_results: int = 3,
        max_chars_per_page: int = 5000,
        region: str = "us-en",
        safesearch: str = "moderate",
        backend: str = "auto"
    ) -> str:
        """
        Search DuckDuckGo and fetch actual webpage content
        
        Args:
            query: The search query
            num_results: Number of results to fetch (default: 3)
            max_chars_per_page: Maximum characters per page (default: 5000)
            region: Region code (default: us-en)
            safesearch: Safe search setting (on/moderate/off, default: moderate)
            backend: Search backend (default: auto, options: bing, brave, google, etc.)
            
        Returns:
            Formatted search results with full content
        """
        try:
            # Step 1: Get search results from DuckDuckGo using ddgs
            ddgs = DDGS(proxy=self.proxy, timeout=self.timeout)
            search_results = ddgs.text(
                query=query,
                region=region,
                safesearch=safesearch,
                max_results=num_results,
                page=1,
                backend=backend
            )
            
            if not search_results:
                return "No results found"
            
            # Step 2: Fetch content from each webpage
            results = []
            
            async with aiohttp.ClientSession() as session:
                for i, result in enumerate(search_results, 1):
                    title = result.get('title', 'No title')
                    link = result.get('href', '')
                    snippet = result.get('body', 'No description')
                    
                    # Fetch the actual webpage
                    try:
                        headers = {
                            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
                        }
                        
                        async with session.get(
                            link,
                            timeout=aiohttp.ClientTimeout(total=10),
                            headers=headers,
                            ssl=False  # Skip SSL verification for problematic sites
                        ) as response:
                            if response.status == 200:
                                html = await response.text()
                                
                                # Parse HTML and extract text
                                soup = BeautifulSoup(html, 'html.parser')
                                
                                # Remove unwanted elements
                                for element in soup(['script', 'style', 'nav', 'footer', 'header', 'aside']):
                                    element.decompose()
                                
                                # Get text content
                                text = soup.get_text(separator=' ', strip=True)
                                
                                # Clean up whitespace
                                lines = (line.strip() for line in text.splitlines())
                                text = ' '.join(line for line in lines if line)
                                
                                # Truncate if too long
                                if len(text) > max_chars_per_page:
                                    text = text[:max_chars_per_page] + "..."
                                
                                results.append(
                                    f"Result {i}:\n"
                                    f"Title: {title}\n"
                                    f"URL: {link}\n"
                                    f"Snippet: {snippet}\n\n"
                                    f"Full Content:\n{text}"
                                )
                            else:
                                results.append(
                                    f"Result {i}:\n"
                                    f"Title: {title}\n"
                                    f"URL: {link}\n"
                                    f"Snippet: {snippet}\n"
                                    f"Note: Could not fetch content (HTTP {response.status})"
                                )
                    except Exception as e:
                        results.append(
                            f"Result {i}:\n"
                            f"Title: {title}\n"
                            f"URL: {link}\n"
                            f"Snippet: {snippet}\n"
                            f"Note: Error fetching content - {str(e)}"
                        )
            
            return "\n\n" + ("="*80 + "\n\n").join(results)
            
        except (DDGSException, RatelimitException, TimeoutException) as e:
            return f"Search error: {str(e)}"
        except Exception as e:
            return f"Unexpected error: {str(e)}"
    
    @kernel_function(
        name="quick_search",
        description="Quick web search using DuckDuckGo (returns only titles and snippets, faster)"
    )
    def quick_search(
        self,
        query: str,
        num_results: int = 5,
        region: str = "us-en",
        safesearch: str = "moderate",
        backend: str = "auto",
        timelimit: Optional[str] = None
    ) -> str:
        """
        Quick search without fetching full content (synchronous)
        
        Args:
            query: The search query
            num_results: Number of results (default: 5)
            region: Region code (default: us-en)
            safesearch: Safe search setting (default: moderate)
            backend: Search backend (default: auto)
            timelimit: Time limit for results: d (day), w (week), m (month), y (year). Defaults to None.
            
        Returns:
            Formatted search results with titles and snippets only
        """
        try:
            ddgs = DDGS(proxy=self.proxy, timeout=self.timeout)
            search_results = ddgs.text(
                query=query,
                region=region,
                safesearch=safesearch,
                timelimit=timelimit,
                max_results=num_results,
                page=1,
                backend=backend
            )
            
            if not search_results:
                return "No results found"
            
            results = []
            for i, result in enumerate(search_results, 1):
                title = result.get('title', 'No title')
                link = result.get('href', '')
                snippet = result.get('body', 'No description')
                
                results.append(
                    f"{i}. {title}\n"
                    f"URL: {link}\n"
                    f"Snippet: {snippet}"
                )
            
            return "\n\n".join(results)
            
        except (DDGSException, RatelimitException, TimeoutException) as e:
            return f"Search error: {str(e)}"
        except Exception as e:
            return f"Unexpected error: {str(e)}"

## Création du prompt de l'agent

Cette section doit contenir le prompt système pour l'agent de calcul de coûts. Le prompt doit définir le rôle de l'agent, expliquer comment il doit utiliser les outils mathématiques pour calculer les volumes/surfaces, rechercher les prix des matériaux sur le web, et fournir une estimation de coût finale à l'utilisateur.

In [ ]:
agent_prompt = """


"""

## Définition de la boucle pour intéragir avec les agents

Cette section doit contenir le code de la boucle d'interaction avec l'agent de calcul de coûts. Similaire à la boucle de l'agent de faisabilité, elle initialise le kernel, enregistre les plugins (mathématiques et recherche web), et permet une conversation interactive où l'utilisateur peut demander des estimations de coûts pour différents projets.

In [ ]:
# Inclure votre code ICI

# Bonus, inclure un MCP à l'agent


Cette section est un exercice bonus pour intégrer un MCP (Model Context Protocol) à l'agent. Le MCP permet d'étendre les capacités de l'agent en lui donnant accès à des outils et contextes supplémentaires via un protocole standardisé.

**Amusez-vous bien !** 🚀